# MVA Cours Numerical PDEs for image analysis

## TP1.1 Sparse automatic differentiation

We implement a automatic differentiation class, and explore some of its applications.

## 0. Importing the required libraries

In [8]:
import numpy as np
import scipy.sparse
from matplotlib import pyplot as plt

## 1. Implementation

In [345]:
class Sparse:
    """
    A class for Sparse, first order, forward automatic differentiation.
    Members : 
    - x : an array of arbitrary shape (n_1,...,n_k)
    - v : an array of shape (size_ad, n_1,...,n_k), where size_ad is arbitrary integer
    - i : an array of shape (size_ad, n_1,...,n_k).
    
    Represents the following Taylor expansion, where h is a symbolic perturbation
    x + sum(h[i[k]]*v[k] for k in range(size_ad)) + o(\|h\|)
    """
    
    def __init__(self,x,v,i):
        self.x = x
        self.v = np.asarray(v) 
        self.i = np.asarray(i)
        assert np.shape(x) == np.shape(v)[1:]
        assert np.shape(v) == np.shape(i)
    
    def __repr__(self):
        return f"Sparse({self.x},{self.v},{self.i})"
    
    @property
    def size_ad(self): return len(self.v)
    @property
    def shape(self): return self.x.shape
    
    def convert(self,other,sshape=None):
        if sshape is None: sshape = self.shape
        if isinstance(other,Sparse): 
            assert other.shape == sshape
            return other
        else: # Broadcast other array, otherwise won't match
            shape_ad = (0,*sshape)
            return Sparse(np.broadcast_to(other,sshape),
                            np.zeros(shape_ad),np.zeros(shape_ad,dtype=int))
    
    # Base arithmetic operators
    def __add__(self,other):
        a,b = self,self.convert(other)
        return Sparse(a.x+b.x, np.concatenate([a.v,b.v]), np.concatenate([a.i,b.i]))
    
    def __sub__(self,other):
        a,b = self,self.convert(other)
        return Sparse(a.x-b.x,np.concatenate([a.v,-b.v]), np.concatenate([a.i,b.i]))
    
    def __mul__(self,other):
        a,b = self,self.convert(other)
        return Sparse(a.x*b.x, np.concatenate([b.x*a.v,a.x*b.v]), np.concatenate([a.i,b.i]))

    def __truediv__(self,other):
        a,b = self,self.convert(other)
        return Sparse(a.x/b.x, np.concatenate([a.v/b.x,- a.x*b.v/b.x**2]), np.concatenate([a.i,b.i]))

    # Other operators
    def __neg__(self): return Sparse(-self.x,-self.v, self.i)
    __radd__ = __add__
    __rmul__ = __mul__
    def __rsub__(self,other): return self.convert(other)-self
    def __rtruediv__(self,other): return self.convert(other)/self
    
    
    # Special functions
    def __pow__(self,r):
        x = self.x
        return Sparse(x**r, r*x**(r-1) * self.v, self.i)
    
    def sqrt(self): 
        s = np.sqrt(self.x)
        return Sparse(s, self.v/(2*s), self.i)
    
    def sin(self):
        s,c = np.sin(self.x),np.cos(self.x)
        return Sparse(s, c*self.v, self.i)
    
    def cos(self):
        s,c = np.sin(self.x),np.cos(self.x)
        return Sparse(c,-s*self.v, self.i)

    # Several dimensions
    def __getitem__(self,key):
        return Sparse(self.x[key],self.v[:,key], self.i[:,key])
    def __setitem__(self,key,other):
        a,b = self,self.convert(other,self.x[key].shape)
        assert a.size_ad>=b.size_ad # Ensure AD data will fit
        a.x[key]   = b.x
        a.v[:b.size_ad,key] = b.v
        a.v[b.size_ad:,key] = 0.
        a.i[:b.size_ad,key] = b.i
    def flatten(self):
        shape_ad = (self.size_ad,-1)
        return Sparse(np.reshape(self.x,-1),np.reshape(self.v,shape_ad), np.reshape(self.i,shape_ad))
    def roll(self,shift,axis=0):
        assert axis>=0
        return Sparse(np.roll(self.x,shift,axis),np.roll(self.v,shift,axis+1),np.roll(self.i,shift,axis+1))
    
    def sum(self,axis):
        assert axis==0
        shape_ad = (self.size_ad*x.shape[0],*x.shape[1:])           
        return Sparse(self.x.sum(axis),self.v.reshape(shape_ad),self.i.reshape(shape_ad))
    
    # Conversion of AD information to a sparse matrix
    def triplets(self):
        assert np.ndim(self.x)==1
        return (self.v.flatten(),(np.tile(np.arange(len(self.x)),self.size_ad), self.i.flatten()))
    def tocsr(self): return scipy.sparse.coo_matrix(self.triplets()).tocsr()
    def spsolve(self): return scipy.sparse.linalg.spsolve(self.tocsr(),-self.x)
        
# Create some sample variables for testing purposes
a = Sparse(1.,[1.,2.],[1,4]) # 1 + (h_1 + 2*h_4) + o(|h|)
b = Sparse(2.,[3.,4.],[0,1]) # 2 + (3*h_0+4*h_1) + o(|h|)

In [346]:
def close_to_zzero(a): 
    """Checks that a Taylor expansion has its zeroth and first order components close to zero."""
    m = a.flatten().tocsr() # Convert AD information to sparse matrix and compress
    return np.allclose(a.x,0) and np.allclose(m.data,0)

### 1.1 String representation

Implement the `Sparse.__repr__` method. It should return a string containing a readable representation of the object, that (ideally) can be used to reconstruct it.

In [120]:
print(a)

Sparse(1.0,[1. 2.],[1 4])


### 1.2 Arithmetic operators

*Task:* Implement the `Sparse.__add__` and `Sparse.__sub__` methods. They must return a `Sparse` object, in accordance with the Taylor expansion rules.
For instance:
$$
    (x+ \sum_{1 \leq k \leq K} v_k h_{i_k} + o(\|h\|)) + (x+ \sum_{1 \leq k \leq K'} v'_k h_{i'_k} + o(\|h\|)) 
    = (x+x') + (\sum_{1 \leq k \leq K} v_k h_{i_k} + \sum_{1 \leq k \leq K'} v'_k h_{i'_k}) + o(\|h\|)
$$

In [100]:
print(f"Addition : {a+b=}")     # Operator + is __add__
print(f"Substraction : {a-b=}") # Operator * is __sub__

Addition : a+b=Sparse(3.0,[1. 2. 3. 4.],[1 4 0 1])
Substraction : a-b=Sparse(-1.0,[ 1.  2. -3. -4.],[1 4 0 1])


In [101]:
assert close_to_zzero(a+b-a-b) # Unit test for checking implementation

*Task:* Implement the `Dense.__mul__` method, which must be in accordance with the following Taylor expansion:
$$
    (x + <v,h> + o(\|h\|)) + (x'+<v',h>+o(\|h\|)) = (x x') + <(x v'+x' v),h> + o(\|h\|)
$$

In [102]:
print(f"Multiplication {a*b=}")  # Operator * is __mul__
print(f"Division {a/b=}")        # Operator / is __truediv__

Multiplication a*b=Sparse(2.0,[2. 4. 3. 4.],[1 4 0 1])
Division a/b=Sparse(0.5,[ 0.5   1.   -0.75 -1.  ],[1 4 0 1])


In [103]:
assert close_to_zzero(a*b/a/b - 1) # Unit test for checking implementation

### 1.3 Other arithmetic operators

**Unary negation**

*Task:* Implement the unary negation `Sparse.__neg__` operator.

In [104]:
print(f"Unary negation {-a=}")

Unary negation -a=Sparse(-1.0,[-1. -2.],[1 4])


In [105]:
assert close_to_zzero(-a+a) # Unit test for checking implementation

**Operators acting on the right**

*Task:* Implement the `__rmul__` and `__rdiv__` operations, similar to the `Dense` case.

In [106]:
print(f"Right multiplication {2*a=}")
print(f"Right division {2/a=}")

Right multiplication 2*a=Sparse(2.0,[2. 4.],[1 4])
Right division 2/a=Sparse(2.0,[-2. -4.],[1 4])


In [107]:
assert close_to_zzero(2./(2./a) - a)

### 1.4 Special functions

Special functions, whose derivative is usually known explicitly, act on Taylor expansions as 
$$
    f(x + <v,h> + o(\|h\|)) = f(x) + <f'(x) v, h> + o(h).
$$

*Task:* Implement the `sqrt` and `sin` methods of the `Dense` class.

In [108]:
print(f"Square root function {np.sqrt(a)=}")
print(f"Sine function {np.sin(a)=}")

Square root function np.sqrt(a)=Sparse(1.0,[0.5 1. ],[1 4])
Sine function np.sin(a)=Sparse(0.8414709848078965,[0.54030231 1.08060461],[1 4])


In [109]:
assert close_to_zzero(np.sqrt(a**2)-a)
assert close_to_zzero(np.sin(np.pi-a) - np.sin(a))

## 2. Application (One dimensional PDEs)

In [347]:
def vector_ad(x):
    assert np.ndim(x)==1
    shape_ad = (1,len(x))
    return Sparse(x,np.ones(shape_ad),np.arange(len(x)).reshape(shape_ad))

In [348]:
u = vector_ad(np.linspace(0,1,10))

In [349]:
u.roll(1,0)

Sparse([1.         0.         0.11111111 0.22222222 0.33333333 0.44444444
 0.55555556 0.66666667 0.77777778 0.88888889],[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]],[[9 0 1 2 3 4 5 6 7 8]])

### 2.1 Solve the Poisson equation

In [350]:
X,dx = np.linspace(-np.pi,np.pi,10,retstep=True)
def f(x): return np.sin(x)

In [351]:
def Poisson(u,f,bc,h):
    residue = (u.roll(1)-2*u+u.roll(-1))/h**2 - f
    residue[0] = u[0]-bc[0]
    residue[-1] = u[-1]-bc[1]
    return residue

In [352]:
u = vector_ad(np.zeros_like(X))
res = Poisson(u,f(X),[1,1],dx)

In [353]:
res.spsolve()

array([ 1.        ,  1.66954359,  2.02580029,  1.90207363,  1.3562567 ,
        0.6437433 ,  0.09792637, -0.02580029,  0.33045641,  1.        ])

In [336]:
res

Sparse([-1.          0.64278761  0.98480775  0.8660254   0.34202014 -0.34202014
 -0.8660254  -0.98480775 -0.64278761 -1.        ],[[ 1.          2.05175397  2.05175397  2.05175397  2.05175397  2.05175397
   2.05175397  2.05175397  2.05175397  1.        ]
 [ 0.         -4.10350794 -4.10350794 -4.10350794 -4.10350794 -4.10350794
  -4.10350794 -4.10350794 -4.10350794  0.        ]
 [ 0.          2.05175397  2.05175397  2.05175397  2.05175397  2.05175397
   2.05175397  2.05175397  2.05175397  0.        ]],[[0 0 1 2 3 4 5 6 7 9]
 [0 1 2 3 4 5 6 7 8 9]
 [1 2 3 4 5 6 7 8 9 0]])

In [318]:
import scipy.sparse.linalg

In [319]:
scipy.sparse.linalg.spsolve(res.tocsr(),res.x)

array([-1.        , -1.66954359, -2.02580029, -1.90207363, -1.3562567 ,
       -0.6437433 , -0.09792637,  0.02580029, -0.33045641, -1.        ])

In [276]:
res[0] = u[0]-1

In [272]:
res.triplets()

(array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2.,  0.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.]),
 (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
         2, 3, 4, 5, 6, 7, 8, 9]),
  array([0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2,
         3, 4, 5, 6, 7, 8, 9, 0])))

In [273]:
res.tocsr().todense()

matrix([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -2.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2.,  1.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2.]])

In [274]:
np.tile(np.arange(5),3)

array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4])

## 2.2 Solve a non-linear PDE